# 0. Load modules

This is quite personal, but I specify modules fully.  
Also, modules can be install either with pip or conda.  
That's all the info you need. Google is your friend.

In [1]:
import cobra, os, importlib, dotenv, numpy
import rich # needs to be version 6.2.0. Consider #!pip install rich==6.2.0

# 1. user-defined variables

In [2]:
working_solver = "cplex" # much faster version of glpk

In [3]:
io_path = '/home/adrian/hub/SystemsBio-group2/yeast-GEM/code/io.py'

# 1. retrieve yeast model

Clone just once.

In [4]:
#! git clone https://github.com/SysBioChalmers/yeast-GEM.git

Here is the nightmare of reading the model using their methods.

In [5]:
os.chdir('yeast-GEM')
#! touch .env

# find .env + define paths:
dotenv_path = dotenv.find_dotenv()
REPO_PATH = os.path.dirname(dotenv_path)
MODEL_PATH = f"{REPO_PATH}/model/yeast-GEM.xml"

In [6]:
spec = importlib.util.spec_from_file_location("alo", io_path)
foo = importlib.util.module_from_spec(spec)
spec.loader.exec_module(foo)
model = foo.read_yeast_model()

Restricted license - for non-production use only - expires 2023-10-25


# 2. explore the model

In [7]:
model.solver = working_solver

In [8]:
model.solver

In [9]:
model.metabolites.get_by_id("s_0681[e]") #s_0681[e] is the id of ethanol in the model

Metabolite identifier,s_0681[e]
Name,ethanol [extracellular]
Memory address,0x07fc15c597e20
Formula,C2H6O
Compartment,e
In 2 reaction(s),"r_1762, r_1761"


In [10]:
model.reactions.get_by_id('r_1761')

Reaction identifier,r_1761
Name,ethanol exchange
Memory address,0x07fc15abd98e0
Stoichiometry,s_0681[e] --> ethanol [extracellular] -->
GPR,
Lower bound,0.0
Upper bound,1000.0


In [11]:
model.objective = "r_1761"

In [12]:
model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
s_0565[e],r_1714,1,6,100.00%
Metabolite,Reaction,Flux,C-Number,C-Flux
s_0458[e],r_1672,-2,1,33.33%
s_0681[e],r_1761,-2,2,66.67%


# 3. run FVA

In [13]:
%%time
cobra.flux_analysis.flux_variability_analysis(model, fraction_of_optimum=0.95)

CPU times: user 204 ms, sys: 245 ms, total: 448 ms
Wall time: 9.36 s


,minimum,maximum
r_0001,0.000000,0.600000
r_0002,0.000000,0.600000
r_0003,-0.054545,0.000000
r_0004,0.000000,0.600000
r_0005,0.000000,0.004913
...,...,...
r_4694,0.000000,0.046154
r_4695,0.000000,0.046154
r_4697,0.000000,0.046154
r_4698,0.000000,0.046154


In [14]:
model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
s_0565[e],r_1714,1,6,100.00%
Metabolite,Reaction,Flux,C-Number,C-Flux
s_0458[e],r_1672,-2,1,33.33%
s_0681[e],r_1761,-2,2,66.67%


In [15]:
model.summary(fva=0.95)

Metabolite,Reaction,Flux,Range,C-Number,C-Flux
s_0565[e],r_1714,1,[0.95; 1],6,100.00%
Metabolite,Reaction,Flux,Range,C-Number,C-Flux
s_0026[e],r_1546,0,[-0.1; 0],3,0.00%
s_0029[e],r_1547,0,[-0.04286; 0],6,0.00%
s_0032[e],r_1548,0,[-0.02609; 0],9,0.00%
s_0036[e],r_1549,0,[-0.05455; 0],4,0.00%
s_0058[e],r_1550,0,[-0.04286; 0],6,0.00%
s_0067[e],r_1552,0,[-0.1; 0],4,0.00%
s_0080[e],r_1553,0,[-0.03158; 0],9,0.00%
s_0163[e],r_1572,0,[-0.04; 0],7,0.00%


In [16]:
info = model.summary(fva=0.95).to_frame()

In [17]:
info.loc['r_1761']

reaction         r_1761
metabolite    s_0681[e]
factor             -1.0
flux               -2.0
minimum            -2.0
maximum            -1.9
Name: r_1761, dtype: object

# 4. run single-gene deletion analysis

In [18]:
%%time
single_deletion_results = cobra.flux_analysis.single_gene_deletion(model)

CPU times: user 16.9 ms, sys: 117 ms, total: 134 ms
Wall time: 670 ms


In [19]:
single_deletion_results

,ids,growth,status
0,{YDL166C},2.000000,optimal
1,{YDR408C},2.000000,optimal
2,{YIL023C},2.000000,optimal
3,{YDR047W},2.000000,optimal
4,{YFR044C},2.000000,optimal
...,...,...,...
1145,{YOR283W},2.000000,optimal
1146,{YKL150W},2.000000,optimal
1147,{YKL060C},1.677994,optimal
1148,{YBR252W},2.000000,optimal


In [20]:
print(numpy.min(single_deletion_results['growth']))
print(numpy.max(single_deletion_results['growth']))

1.6779939709509457
2.0


# 5. run double KO

In [21]:
# def filter_unwanted_reactions_from_model(model):
#   reactions_to_remove = [] #Clean list
#   for ele in range(0,len(model.reactions)):
#     if len(model.reactions[ele].compartments) > 1:
#       reactions_to_remove.append(model.reactions[ele].id)
#     elif model.reactions[ele] in model.exchanges:
#       reactions_to_remove.append(model.reactions[ele].id)
#     else:
#       model2 = model.copy()
#       model2.reactions.get_by_id(model.reactions[ele].id).bounds = (0,0)
#       sol_tmp = model2.slim_optimize()
#       if sol_tmp < 1e-4:
#         reactions_to_remove.append(model.reactions[ele].id)
#   model3 = model.copy()
#   model3.remove_reactions(reactions_to_remove) #Remove those reactions
#   return model3

In [22]:
# model_copy = filter_unwanted_reactions_from_model(model) #Running this took up all the RAM and didn't finish running on my laptop

In [23]:
print(len(model.genes))

1150


Working on an environment of 20 threads (Intel® Core™ i9-10900 CPU @ 2.80GHz × 20)  

Gene pair KOs using gplk:  

125 --> 13 sec  
250 --> 41 sec  
500 --> 120 sec

Gene pair KOs using cplex: 

125 --> 10 sec  
250 --> 33 sec  
500 --> 78 sec  
all --> 6 min  

In [24]:
%%time
doubleKO_solutions = cobra.flux_analysis.double_gene_deletion(model, model.genes[:125])

CPU times: user 45.5 ms, sys: 129 ms, total: 174 ms
Wall time: 10.3 s


In [25]:
print(numpy.min(doubleKO_solutions['growth']))
print(numpy.max(doubleKO_solutions['growth']))

1.666666666667311
2.000000000006594


In [26]:
%%time
doubleKO_solutions = cobra.flux_analysis.double_gene_deletion(model, model.genes[:250])

CPU times: user 257 ms, sys: 133 ms, total: 390 ms
Wall time: 31 s


In [27]:
print(numpy.min(doubleKO_solutions['growth']))
print(numpy.max(doubleKO_solutions['growth']))

0.6403853278993097
2.000000000150294


In [28]:
%%time
doubleKO_solutions = cobra.flux_analysis.double_gene_deletion(model, model.genes[:500])

CPU times: user 1.31 s, sys: 208 ms, total: 1.51 s
Wall time: 1min 20s


In [29]:
print(numpy.min(doubleKO_solutions['growth']))
print(numpy.max(doubleKO_solutions['growth']))

0.6403853278992528
2.0000000000579803


In [30]:
%%time
doubleKO_solutions = cobra.flux_analysis.double_gene_deletion(model)

CPU times: user 8.2 s, sys: 445 ms, total: 8.64 s
Wall time: 5min 52s


In [31]:
print(numpy.min(doubleKO_solutions['growth']))
print(numpy.max(doubleKO_solutions['growth']))

0.6403853279000487
2.0000000001839453
